In [ ]:
import torch

weights1 = torch.tensor([[0.1, 0.2],
                        [0.3, 0.4]], dtype=torch.float32, requires_grad=True)
inputs   = torch.tensor([[0.05, 0.15],
                        [0.25, 0.35]], dtype=torch.float32, requires_grad=True)
weights2 = torch.tensor([[0.01, 0.02],
                        [0.03, 0.04]], dtype=torch.float32, requires_grad=True)

layer1_linear = weights1.matmul(inputs.transpose(0,1))
layer1_activated = torch.relu(layer1_linear)

layer2_linear = layer1_activated.matmul(weights2)
layer2_sum = layer2_linear.sum(1, keepdim=True)  # Keep dimension to make it [2,1] instead of [2]
layer2_output = layer2_sum

expand_tensor = torch.tensor([[1.0, 1.0]], dtype=torch.float32)
layer2_output_expanded = layer2_output.matmul(expand_tensor)

layer3_linear = layer2_output_expanded * weights1
layer3_nonlinear = torch.exp(layer3_linear)
layer3_output = layer3_nonlinear + layer2_output_expanded

logits = layer3_output.matmul(weights2.transpose(0,1))

probabilities = 1.0 / (1.0 + torch.exp(-1.0 * logits))

epsilon = 1e-10
log_probs = torch.log(probabilities + epsilon)
neg_log_probs = -1.0 * log_probs + 0.1

loss_sum = neg_log_probs.sum(0).sum(0)

loss_sum.backward()

print("Gradient of weights1:")
print(weights1.grad)
print("\nGradient of inputs:")
print(inputs.grad)
print("\nGradient of weights2:")
print(weights2.grad)

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

# --- Helper functions to simulate CSV reading and flattening ---

def read_csv(file_path):
    # For demonstration purposes, we simulate reading MNIST-like data.
    # Let's assume we have 1200 examples, each with 784 features.
    # And the labels are one-hot encoded vectors of length 10.
    num_examples = 1200
    num_features = 784
    num_classes = 10

    # Simulated input data: shape (1200, 784)
    data = np.random.rand(num_examples, num_features).astype(np.float32)
    # Simulated labels: create one-hot encoded labels
    labels = np.zeros((num_examples, num_classes), dtype=np.float32)
    random_class = np.random.randint(0, num_classes, size=num_examples)
    labels[np.arange(num_examples), random_class] = 1.0

    return data, labels

def flatten(arr):
    # In Python with numpy, flattening is straightforward
    return arr.flatten()

# --- Read data ---
data_np, labels_np = read_csv("data/mnist_dummy.csv")

# In your C++ code the inputs tensor shape was {784, 1200} then transposed to (1200,784).
# In PyTorch it's simpler: we make 'inputs' of shape (1200,784) and 'labels' of shape (1200,10).
inputs = torch.tensor(data_np)       # shape: (1200, 784)
labels = torch.tensor(labels_np)       # shape: (1200, 10)

# --- Initialize weights ---
# weights1 shape: (784, 256), weights2 shape: (256, 10)
# They are initialized with small random numbers: scaled by 0.01.
weights1 = (torch.rand(784, 256) * 0.01).requires_grad_(True)
weights2 = (torch.rand(256, 10) * 0.01).requires_grad_(True)

learning_rate = 0.0001

# --- Training Loop ---
for epoch in range(10):
    # Forward pass
    # Equivalent to: 
    #    a1 = inputs.transpose().matmul(weights1) in the C++ code,
    # here we use inputs (shape: 1200x784) directly:
    a1 = inputs.matmul(weights1)   # (1200,784) @ (784,256) = (1200,256)
    z1 = a1.clamp(min=0)             # ReLU activation

    a2 = z1.matmul(weights2)         # (1200,256) @ (256,10) = (1200,10)

    # Compute softmax manually:
    exp_a2 = torch.exp(a2)
    # The denominator: sum over classes for each example.
    # Adding a small epsilon (1e-10) for numerical stability.
    denom = exp_a2.sum(dim=1, keepdim=True) + 1e-10  # shape: (1200,1)
    z2 = exp_a2 / denom            # Broadcasting to (1200,10)

    # Compute the cross-entropy loss manually.
    # In the original code, loss is computed as:
    #   loss = ((-1.0) * log(z2) * labels.transpose()).sum(0).sum(1)
    # In our case, labels are (1200, 10) so we can compute the elementwise product
    loss = (-1.0 * torch.log(z2 + 1e-10) * labels).sum()  # Scalar loss
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

    # Backward pass: compute gradients with respect to weights.
    loss.backward()

    # Update weights using a manual gradient descent step.
    with torch.no_grad():
        weights1 -= learning_rate * weights1.grad
        weights2 -= learning_rate * weights2.grad

        # Zero the gradients after updating
        weights1.grad.zero_()
        weights2.grad.zero_()


In [ ]:
import torch

# Create two 2x2 tensors with gradients enabled.
a = torch.tensor([[1.0, 2.0],
                  [3.0, 4.0]], requires_grad=True)
b = torch.tensor([[0.1, 0.2],
                  [0.3, 0.4]], requires_grad=True)

# --- Test 1: Sum ---
# Sum over dimension 0 (i.e. summing rows) and keep the dimension.
sum_result = a.sum(dim=0, keepdim=True)  
print(sum_result)
# Because sum_result is not scalar (it's shape [1,2]),
# we pass an explicit gradient tensor of matching shape.
sum_result.backward(torch.ones_like(sum_result))
print("Sum gradient:")
print(a.grad)
# Reset a's gradients.
a.grad.zero_()

# --- Test 2: Matmul ---
matmul_result = a.matmul(b)  # Matmul returns a [2,2] tensor.
matmul_result.backward(torch.ones_like(matmul_result))
print("\nMatmul gradient for a:")
print(a.grad)
print("\nMatmul gradient for b:")
print(b.grad)
a.grad.zero_()
b.grad.zero_()

# --- Test 3: Transpose ---
# Compute the transpose of 'a'.
transpose_result = a.transpose(0, 1)  # Still 2x2.
transpose_result.backward(torch.ones_like(transpose_result))
print("\nTranspose gradient:")
print(a.grad)
a.grad.zero_()

# --- Test 4: Exponential ---
exp_result = a.exp()  # Elementwise exponentiation.
exp_result.backward(torch.ones_like(exp_result))
print("\nExp gradient:")
print(a.grad)


In [ ]:
import torch

weights1 = torch.tensor([[0.1, 0.3],
                        [0.2, 0.4]], dtype=torch.float32, requires_grad=True)
inputs   = torch.tensor([[0.05, 0.25],
                        [0.15, 0.35]], dtype=torch.float32, requires_grad=True)
weights2 = torch.tensor([[0.01, 0.03],
                        [0.02, 0.04]], dtype=torch.float32, requires_grad=True)

layer1_linear = weights1.matmul(inputs.transpose(0,1))
layer1_activated = torch.relu(layer1_linear)

layer2_linear = layer1_activated.matmul(weights2)
layer2_sum = layer2_linear.sum(1, keepdim=True)  # Keep dimension to make it [2,1] instead of [2]
layer2_output = layer2_sum

expand_tensor = torch.tensor([[1.0, 1.0]], dtype=torch.float32)
layer2_output_expanded = layer2_output.matmul(expand_tensor)

layer3_linear = layer2_output_expanded * weights1
layer3_nonlinear = torch.exp(layer3_linear)
layer3_output = layer3_nonlinear + layer2_output_expanded

logits = layer3_output.matmul(weights2.transpose(0,1))

probabilities = 1.0 / (1.0 + torch.exp(-1.0 * logits))

epsilon = 1e-10
log_probs = torch.log(probabilities + epsilon)
neg_log_probs = -1.0 * log_probs + 0.1

loss_sum = neg_log_probs.sum(0).sum(0)

loss_sum.backward()

print("Gradient of weights1:")
print(weights1.grad)
print("\nGradient of inputs:")
print(inputs.grad)
print("\nGradient of weights2:")
print(weights2.grad)

In [ ]:
import torch

weights1 = torch.tensor([[0.1, 0.3],
                        [0.2, 0.4]], dtype=torch.float32, requires_grad=True)
inputs = torch.tensor([[0.05, 0.25],
                      [0.15, 0.35]], dtype=torch.float32)
weights2 = torch.tensor([[0.01, 0.03],
                        [0.02, 0.04]], dtype=torch.float32)

layer1 = weights1.matmul(inputs.transpose(0,1))
layer1.retain_grad()
layer1_activated = torch.relu(layer1)
layer1_activated.retain_grad()

# print(layer1_activated)
# print(weights2)

layer2 = layer1_activated.matmul(weights2) # HERE
# print(layer2)
layer2.retain_grad()

layer2_sum = layer2.sum(1, keepdim=True)
layer2_sum.retain_grad()
layer2_expanded = layer2_sum.matmul(torch.tensor([[1.0, 1.0]], dtype=torch.float32))
layer2_expanded.retain_grad()

layer3 = layer2_expanded * weights1  # Element-wise multiplication
layer3.retain_grad()
loss = layer3.sum()


print("of weights1:")
print(weights1)

print("\nof layer1:")
print(layer1)

print("\nof layer1_activated:")
print(layer1_activated)

print("\nof layer2:")
print(layer2)

print("\nof layer2_sum:")
print(layer2_sum)

print("\nof layer2_expanded:")
print(layer2_expanded)

print("\nof layer3:")
print(layer3)


loss.backward()

print("Gradient of weights1:")
print(weights1.grad)

print("\nGradient of layer1:")
print(layer1.grad)

print("\nGradient of layer1_activated:")
print(layer1_activated.grad)

print("\nGradient of layer2:")
print(layer2.grad)

print("\nGradient of layer2_sum:")
print(layer2_sum.grad)

print("\nGradient of layer2_expanded:")
print(layer2_expanded.grad)

print("\nGradient of layer3:")
print(layer3.grad)


'''
Gradient of weights1: 
Tensor(data=[0.0146, 0.0214, 0.0226, 0.0334], shape=[2, 2])

Gradient of layer1: HERE!
Tensor(data=[0.012, 0.018, 0.028, 0.042], shape=[2, 2])

Gradient of layer1_activated:
Tensor(data=[0.012, 0.018, 0.028, 0.042], shape=[2, 2])

Gradient of layer2: HERE!
Tensor(data=[0.4, 0.6, 0.4, 0.6], shape=[2, 2])

Gradient of layer2_sum: 
Tensor(data=[0.4, 0.6], shape=[2, 1])

Gradient of layer2_expanded:
Tensor(data=[0.1, 0.2, 0.3, 0.4], shape=[2, 2])

Gradient of layer3:
Tensor(data=[1, 1, 1, 1], shape=[2, 2])
'''

In [ ]:

# weights2 = torch.tensor([[0.01, 0.02], [0.03, 0.04]], requires_grad=True)
# inputs = torch.tensor([[0.05, 0.15], [0.25, 0.35]], requires_grad=True)
# weights1 = torch.tensor([[0.1, 0.2], [0.3, 0.4]], requires_grad=True)

# weights1 = torch.tensor([[0.1, 0.3], [0.2, 0.4]], requires_grad=True)
# inputs = torch.tensor([[0.05, 0.25], [0.15, 0.35]], requires_grad=True)
# weights2 = torch.tensor([[0.01, 0.03], [0.02, 0.04]], requires_grad=True)

# print(weights1)
# print(inputs.T)
# layer1 = weights1.matmul(inputs.T)
# print(layer1)
# print("---")

# print(layer1)
# print(layer2)
# print(layer1.matmul(weights2))

In [ ]:
# WEIGHTS1: Variable(requires_grad=1, data=Tensor(data=[0.1, 0.2, 0.3, 0.4], shape=[2, 2]), grad=Tensor(data=[0, 0, 0, 0], shape=[2, 2]), grad_fn=0)
# INPUT: Variable(requires_grad=1, data=Tensor(data=[0.05, 0.25, 0.15, 0.35], shape=[2, 2]), grad=Tensor(data=[0, 0, 0, 0], shape=[2, 2]), grad_fn=0x565f3ddb9c60)
# LAYER1: Variable(requires_grad=1, data=Tensor(data=[0.035, 0.075, 0.095, 0.215], shape=[2, 2]), grad=Tensor(data=[0, 0, 0, 0], shape=[2, 2]), grad_fn=0x565f3ddb9e10)
# ---
# LAYER1: Variable(requires_grad=1, data=Tensor(data=[0.035, 0.075, 0.095, 0.215], shape=[2, 2]), grad=Tensor(data=[0, 0, 0, 0], shape=[2, 2]), grad_fn=0x565f3ddb9e10)
# WEIGHTS2: Variable(requires_grad=1, data=Tensor(data=[0.01, 0.02, 0.03, 0.04], shape=[2, 2]), grad=Tensor(data=[0, 0, 0, 0], shape=[2, 2]), grad_fn=0)
# LAYER2: Variable(requires_grad=1, data=Tensor(data=[0.0026, 0.0074, 0.0037, 0.0105], shape=[2, 2]), grad=Tensor(data=[0, 0, 0, 0], shape=[2, 2]), grad_fn=0x565f3ddb9fc0)
